In [1]:
import argparse
import math
import h5py
import numpy as np
import tensorflow as tf
import socket
import importlib
import os
import sys
# BASE_DIR = os.path.dirname(os.path.abspath('.'))
BASE_DIR = os.path.abspath('.')
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(os.path.join(BASE_DIR, 'utils'))
import provider
import tf_util

/home/f1001/.pyenv/versions/2.7.14/envs/pointnet/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [24]:
NUM_POINT = 1024

In [25]:
# ModelNet40 official train/test split
TRAIN_FILES = provider.getDataFiles( \
    os.path.join(BASE_DIR, 'data/modelnet40_ply_hdf5_2048/train_files.txt'))
TEST_FILES = provider.getDataFiles(\
    os.path.join(BASE_DIR, 'data/modelnet40_ply_hdf5_2048/test_files.txt'))

In [45]:
current_data, current_label = provider.loadDataFile(TRAIN_FILES[train_file_idxs[0]])
current_data = current_data[:,0:NUM_POINT,:]
current_data, current_label, _ = provider.shuffle_data(current_data, np.squeeze(current_label))            
current_label = np.squeeze(current_label)

In [51]:
current_data.shape

(2048, 1024, 3)

In [54]:
current_label

array([14, 36, 25, ...,  4, 17, 28], dtype=uint8)

In [46]:
# Shuffle train files
train_file_idxs = np.arange(0, len(TRAIN_FILES))
np.random.shuffle(train_file_idxs)

In [21]:
types = []
for lable in current_label:
    if lable not in types:
        types.append(lable)
len(types)

40

In [30]:
train_file_idxs

array([0, 3, 2, 1, 4])

In [31]:
np.random.shuffle(train_file_idxs)
train_file_idxs

array([0, 4, 2, 3, 1])

In [12]:
type(current_data)

numpy.ndarray

In [2]:
!tar -zxvf ~/Downloads/cudnn-8.0-linux-x64-v5.1.tgz

cuda/include/cudnn.h
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.5
cuda/lib64/libcudnn.so.5.1.10
cuda/lib64/libcudnn_static.a


In [6]:
#
# This examaple creates an HDF5 file dset.h5 and an empty datasets /dset in it.
#
import h5py
#
# Create a new file using defaut properties.
#
file = h5py.File('dset.h5','w')
#
# Create a dataset under the Root group.
#
dataset = file.create_dataset("dset",(4, 6), h5py.h5t.STD_I32BE)
print "Dataset dataspace is", dataset.shape
print "Dataset Numpy datatype is", dataset.dtype
print "Dataset name is", dataset.name
print "Dataset is a member of the group", dataset.parent
print "Dataset was created in the file", dataset.file
#
# Close the file before exiting
#
file.close()

Dataset dataspace is (4, 6)
Dataset Numpy datatype is >i4
Dataset name is /dset
Dataset is a member of the group <HDF5 group "/" (1 members)>
Dataset was created in the file <HDF5 file "dset.h5" (mode r+)>


## TRAIN FLUID

### Load Data

#### load_data.py content

In [2]:
%%writefile load_data.py

import argparse
import time
import operator
import os
import sys

import h5py
from nose.tools import assert_equal
import numpy as np
import pandas as pd

BASE_DIR = '/data/datasets/simulation_data'

DATA_DIR = os.path.join(BASE_DIR, 'water/0')
if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)

def get_data_files():
    filenames = []
    # TODO multi folders for different frames
    allfiles = os.listdir(DATA_DIR)
    for filename in allfiles:
        if filename.endswith('.csv'): 
            filenames.append(filename)
    return map(lambda x: os.path.join(DATA_DIR, x), filenames)

def convert_str_float(frame_particles):
    fps = pd.DataFrame(frame_particles[1:], columns=frame_particles[0])
    fps = fps[fps.columns[:-1]]
    for col in fps.columns: 
        #if col == 'isFluidSolid':
        fps[col] = fps[col].astype(float)
    return fps

def laod_csv(filename):
    frame_particles = np.loadtxt(
            filename, dtype=np.str, delimiter=",")
    return convert_str_float(frame_particles)

def load_data_file(filename):
    return laod_csv(filename)

def load_data_label(filename):
    particles = laod_csv(filename)
    cols = particles.columns
    data_cols = operator.add(list(cols[0:6]), list(cols[7:9]))
    label_cols = cols[15:18]
    data = particles[data_cols]
    label = particles[label_cols]
    return data, label

def shuffle_data(data, labels):
    """ Shuffle data and labels.
        Input:
          data: B,N,... numpy array
          label: B,N... numpy array
        Return:
          shuffled data, label and shuffle indices
    """
    idx = np.arange(labels.shape[0])
    np.random.shuffle(idx)
    return data[idx, ...], labels[idx, ...], idx

# Shuffle train files
def concat_data_label(train_files, max_points, dimention_data, dimention_label):
    TRAIN_FILES = train_files
    train_file_idxs = np.arange(0, len(TRAIN_FILES))
    np.random.shuffle(train_file_idxs)
    def get_array(shape):
        return np.empty(shape=shape)
    FRAMES_NUM = len(TRAIN_FILES)
    MAX_POINTS = max_points
    DIMENTION_DATA = dimention_data
    DIMENTION_LABEL = dimention_label
    data_shape = (FRAMES_NUM, MAX_POINTS, DIMENTION_DATA)
    """
    Different from the classification task, our lable is for every particle, we record label with (frame, particle index \
    , three-dimentions accelaration)(BxNx3)
    """
    label_shape = (FRAMES_NUM, MAX_POINTS, DIMENTION_LABEL)
    current_data = get_array(data_shape)
    current_label = get_array(label_shape)
    start = time.clock()
    for fn in range(len(TRAIN_FILES)):
        current_data_single, current_label_single = load_data_label(TRAIN_FILES[train_file_idxs[fn]])
        current_data[fn] = current_data_single.values[:MAX_POINTS, :]
        current_label[fn]= current_label_single.values[:MAX_POINTS, :]
    running = time.clock() - start
    print "runtime: %s" % str(running)
    return current_data, current_label

def jitter_point_cloud(batch_data, sigma=0.01, clip=0.05):
    """ Randomly jitter points. jittering is per point.
        Input:
          BxNx3 array, original batch of point clouds
        Return:
          BxNx3 array, jittered batch of point clouds
    """
    B, N, C = batch_data.shape
    assert(clip > 0)
    jittered_data = np.clip(sigma * np.random.randn(B, N, C), -1*clip, clip)
    jittered_data += batch_data
    return jittered_data

Overwriting load_data.py


#### load_data.py test

In [120]:
run -i load_data.py

In [99]:
TRAIN_FILES = list(get_data_files())[:10]

In [100]:
data, label = concat_data_label(TRAIN_FILES, 10000, 8, 3)

runtime: 15.80032


In [104]:
label.shape

(10, 10000, 3)

In [110]:
data_s, label_s, index = shuffle_data(data, label)

In [122]:
jitter_data = jitter_point_cloud(data_s)

## network

In [3]:
%%writefile train_fluid.py

import argparse
import math
import h5py
import numpy as np
import tensorflow as tf
import socket
import importlib
import os
import sys
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(os.path.join(BASE_DIR, 'utils'))
# import provider
import load_data
import tf_util

parser = argparse.ArgumentParser()
parser.add_argument('--gpu', type=int, default=0, help='GPU to use [default: GPU 0]')
parser.add_argument('--model', default='pointnet_cls', help='Model name: pointnet_cls or pointnet_cls_basic [default: pointnet_cls]')
parser.add_argument('--log_dir', default='log', help='Log dir [default: log]')
parser.add_argument('--num_point', type=int, default=10000, help='Point Number [256/512/1024/2048] [default: 1024]')
parser.add_argument('--max_epoch', type=int, default=250, help='Epoch to run [default: 250]')
parser.add_argument('--batch_size', type=int, default=2, help='Batch Size during training [default: 32]')
parser.add_argument('--learning_rate', type=float, default=0.001, help='Initial learning rate [default: 0.001]')
parser.add_argument('--momentum', type=float, default=0.9, help='Initial learning rate [default: 0.9]')
parser.add_argument('--optimizer', default='adam', help='adam or momentum [default: adam]')
parser.add_argument('--decay_step', type=int, default=200000, help='Decay step for lr decay [default: 200000]')
parser.add_argument('--decay_rate', type=float, default=0.7, help='Decay rate for lr decay [default: 0.8]')
FLAGS = parser.parse_args()


BATCH_SIZE = FLAGS.batch_size
NUM_POINT = FLAGS.num_point
MAX_EPOCH = FLAGS.max_epoch
BASE_LEARNING_RATE = FLAGS.learning_rate
GPU_INDEX = FLAGS.gpu
MOMENTUM = FLAGS.momentum #
OPTIMIZER = FLAGS.optimizer
DECAY_STEP = FLAGS.decay_step #
DECAY_RATE = FLAGS.decay_rate #

MODEL = importlib.import_module(FLAGS.model) # import network module
MODEL_FILE = os.path.join(BASE_DIR, 'models', FLAGS.model+'.py')
LOG_DIR = FLAGS.log_dir
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
os.system('cp %s %s' % (MODEL_FILE, LOG_DIR)) # bkp of model def
os.system('cp train.py %s' % (LOG_DIR)) # bkp of train procedure
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')
LOG_FOUT.write(str(FLAGS)+'\n')

MAX_NUM_POINT = 2048 # TODO the count of particles
NUM_CLASSES = 40 # the outputs of network

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

HOSTNAME = socket.gethostname()

# TODO load sample files
TRAIN_FILES = load_data.get_data_files()[:5]
TEST_FILES = load_data.get_data_files()[5:6]

def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)


def get_learning_rate(batch):
    learning_rate = tf.train.exponential_decay(
                        BASE_LEARNING_RATE,  # Base learning rate.
                        batch * BATCH_SIZE,  # Current index into the dataset.
                        DECAY_STEP,          # Decay step.
                        DECAY_RATE,          # Decay rate.
                        staircase=True)
    learning_rate = tf.maximum(learning_rate, 0.00001) # CLIP THE LEARNING RATE!
    return learning_rate        

def get_bn_decay(batch):
    bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*BATCH_SIZE,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
    bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
    return bn_decay

def train():
    with tf.Graph().as_default():
        with tf.device('/gpu:'+str(GPU_INDEX)):
            pointclouds_pl, labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
            is_training_pl = tf.placeholder(tf.bool, shape=()) # TODO what is 'is_training_pl' for?
            print(is_training_pl)
            
            # Note the global_step=batch parameter to minimize. 
            # That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
            batch = tf.Variable(0)
            bn_decay = get_bn_decay(batch)
            tf.summary.scalar('bn_decay', bn_decay)

            # Get model and loss 
            pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl, bn_decay=bn_decay)
            loss = MODEL.get_loss(pred, labels_pl, end_points) # TODO change to regression loss
            tf.summary.scalar('loss', loss)

#             correct = tf.equal(tf.argmax(pred, 1), tf.to_int64(labels_pl))
#             accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(BATCH_SIZE)
#             tf.summary.scalar('accuracy', accuracy)
#             # end loss

            # Get training operator
            learning_rate = get_learning_rate(batch)
            tf.summary.scalar('learning_rate', learning_rate)
            if OPTIMIZER == 'momentum':
                optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=MOMENTUM)
            elif OPTIMIZER == 'adam':
                optimizer = tf.train.AdamOptimizer(learning_rate)
            train_op = optimizer.minimize(loss, global_step=batch)
            
            # Add ops to save and restore all the variables.
            saver = tf.train.Saver()
            
        # Create a session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        config.allow_soft_placement = True
        config.log_device_placement = False
        sess = tf.Session(config=config)

        # Add summary writers
        #merged = tf.merge_all_summaries()
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'),
                                  sess.graph)
        test_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'))

        # Init variables
        init = tf.global_variables_initializer()
        # To fix the bug introduced in TF 0.12.1 as in
        # http://stackoverflow.com/questions/41543774/invalidargumenterror-for-tensor-bool-tensorflow-0-12-1
        #sess.run(init)
        sess.run(init, {is_training_pl: True})

        ops = {'pointclouds_pl': pointclouds_pl,
               'labels_pl': labels_pl,
               'is_training_pl': is_training_pl,
               'pred': pred,
               'loss': loss,
               'train_op': train_op,
               'merged': merged,
               'step': batch}

        for epoch in range(MAX_EPOCH): # same data with different order
            log_string('**** EPOCH %03d ****' % (epoch))
            sys.stdout.flush()
             
            train_one_epoch(sess, ops, train_writer)
            eval_one_epoch(sess, ops, test_writer)
            
            # Save the variables to disk.
            if epoch % 10 == 0:
                save_path = saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"))
                log_string("Model saved in file: %s" % save_path)



def train_one_epoch(sess, ops, train_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    current_data, current_label = load_data.concat_data_label(TRAIN_FILES, NUM_POINT, 8, 3)
    
    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE
    print('num_batches:', num_batches)
    
    total_correct = 0
    total_seen = 0
    loss_sum = 0
       
    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE

        # Augment batched point clouds by rotation and jittering 
        # TODO AM I also need to do this?
#         rotated_data = provider.rotate_point_cloud(current_data[start_idx:end_idx, :, :])
        jittered_data = load_data.jitter_point_cloud(current_data[start_idx:end_idx, :, :])
        feed_dict = {ops['pointclouds_pl']: jittered_data,
                     ops['labels_pl']: current_label[start_idx:end_idx, :, :],
                     ops['is_training_pl']: is_training,}
        summary, step, _, loss_val, pred_val = sess.run([ops['merged'], ops['step'],
            ops['train_op'], ops['loss'], ops['pred']], feed_dict=feed_dict)
        train_writer.add_summary(summary, step)
#         pred_val = np.argmax(pred_val, 1) # The second para for axis 0 for col and 1 for row ,return index
#         correct = np.sum(pred_val == current_label[start_idx:end_idx])
#         total_correct += correct
#         total_seen += BATCH_SIZE
        loss_sum += loss_val

    log_string('mean loss: %f' % (loss_sum / float(num_batches)))
#     log_string('accuracy: %f' % (total_correct / float(total_seen)))

        
def eval_one_epoch(sess, ops, test_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    total_correct = 0
    total_seen = 0
    loss_sum = 0
#     total_seen_class = [0 for _ in range(NUM_CLASSES)]
#     total_correct_class = [0 for _ in range(NUM_CLASSES)]
    
    MAX_POINTS = 10000
    current_data, current_label = load_data.concat_data_label(TRAIN_FILES, MAX_POINTS, 8, 3)
    
    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE
        
    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE

        feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx, :, :],
                     ops['labels_pl']: current_label[start_idx:end_idx],
                     ops['is_training_pl']: is_training}
        summary, step, loss_val, pred_val = sess.run([ops['merged'], ops['step'],
            ops['loss'], ops['pred']], feed_dict=feed_dict)
#         pred_val = np.argmax(pred_val, 1)
#         correct = np.sum(pred_val == current_label[start_idx:end_idx])
#         total_correct += correct
        total_seen += BATCH_SIZE
        loss_sum += (loss_val*BATCH_SIZE)
#         for i in range(start_idx, end_idx):
#             l = current_label[i]
#             total_seen_class[l] += 1
#             total_correct_class[l] += (pred_val[i-start_idx] == l)
            
    log_string('eval mean loss: %f' % (loss_sum / float(total_seen)))
#     log_string('eval accuracy: %f'% (total_correct / float(total_seen)))
#     log_string('eval avg class acc: %f' % (np.mean(np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float))))
         


if __name__ == "__main__":
    train()
    LOG_FOUT.close()


Writing train_fluid.py


In [7]:
!git add train_fluid.py

In [8]:
!git ci -m "change format of training data "

[master 73bd6d7] change format of training data
 1 file changed, 254 insertions(+)
 create mode 100644 train_fluid.py


## Models

### pointnet_cls.py

In [43]:
%%writefile models/pointnet_cls.py
import tensorflow as tf
import numpy as np
import math
import sys
import os
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, '../utils'))
import tf_util
from transform_nets import input_transform_net, feature_transform_net

def placeholder_inputs(batch_size, num_point):
    pointclouds_pl = tf.placeholder(tf.float32, shape=(batch_size, num_point, 8))
    labels_pl = tf.placeholder(tf.int32, shape=(batch_size, num_point, 3))
    return pointclouds_pl, labels_pl


def get_model(point_cloud, is_training, bn_decay=None):
    """ Classification PointNet, input is BxNx3, output Bx40 """
    batch_size = point_cloud.get_shape()[0].value
    num_point = point_cloud.get_shape()[1].value
    end_points = {}
    # The first transform is for input
    with tf.variable_scope('transform_net1') as sc:
        transform = input_transform_net(point_cloud, is_training, bn_decay, K=10) # output dimentions after fransform
    point_cloud_transformed = tf.matmul(point_cloud, transform)
    # BxNx10
    input_image = tf.expand_dims(point_cloud_transformed, -1)
    # BxNx10x1
    # the second parameter is for the output number of feature maps(FM) not the size of FM
    net = tf_util.conv2d(input_image, 64, [1,10],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv1', bn_decay=bn_decay)
    # BxNx1x64 ,in other words,the size of FM is Nx1 and 64 FMs
    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv2', bn_decay=bn_decay)
    # BxNx1x64
    with tf.variable_scope('transform_net2') as sc:
        transform = feature_transform_net(net, is_training, bn_decay, K=64)
    end_points['transform'] = transform
    # after squeeze BxNx64 ,and we know the shape of transform is Bx64x64, so we get the feature transform result BxNx64
    net_transformed = tf.matmul(tf.squeeze(net, axis=[2]), transform) 
    # BxNx64
    net_transformed = tf.expand_dims(net_transformed, [2])
    
    point_feat = net_transformed # TODO record point-wise feature
    # BxNx1x64
    net = tf_util.conv2d(net_transformed, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv3', bn_decay=bn_decay)
    # BxNx1x64
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv4', bn_decay=bn_decay)
    # BxNx1x128
    net = tf_util.conv2d(net, 1024, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv5', bn_decay=bn_decay)
    # BxNx1x1024

    # Symmetric function: max pooling
    net = tf_util.max_pool2d(net, [num_point,1],
                             padding='VALID', scope='maxpool')

    # Bx1x1x1024  Global Feature TODO the dimentions of GF
    global_feat = net
    global_feat_expand = tf.tile(global_feat, [1, num_point, 1, 1])
    print(global_feat_expand)
    print(point_feat)
    concat_feat = tf.concat([point_feat, global_feat_expand], 3)
    
#     net = tf.reshape(net, [batch_size, -1])
#     # Bx1024
#     net = tf_util.fully_connected(net, 512, bn=True, is_training=is_training,
#                                   scope='fc1', bn_decay=bn_decay)
#     net = tf_util.dropout(net, keep_prob=0.7, is_training=is_training,
#                           scope='dp1')
#     net = tf_util.fully_connected(net, 256, bn=True, is_training=is_training,
#                                   scope='fc2', bn_decay=bn_decay)
#     net = tf_util.dropout(net, keep_prob=0.7, is_training=is_training,
#                           scope='dp2')
#     net = tf_util.fully_connected(net, 40, activation_fn=None, scope='fc3')
    # input BxNx1x(64+1024)
    net = tf_util.conv2d(concat_feat, 512, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv6', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 256, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv7', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv8', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 32, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv9', bn_decay=bn_decay)

    net = tf_util.conv2d(net, 3, [1,1],
                         padding='VALID', stride=[1,1], activation_fn=None,
                         scope='conv10')
    print(net.shape)
    net = tf.squeeze(net, [2]) # BxNxC
    print(net.shape)
    return net, end_points


def get_loss(pred, label, end_points, reg_weight=0.001):
    """ pred: B*NUM_CLASSES
        label: B,
        for fluid pred BxNx3,label BxNx3
    """
    loss = tf.losses.mean_squared_error(label, pred)
    regression_loss = tf.reduce_sum(loss)
    print(regression_loss)
    tf.summary.scalar('classify loss', regression_loss)
#     loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=label)
#     classify_loss = tf.reduce_mean(loss)
#     tf.summary.scalar('classify loss', classify_loss)

    # Enforce the transformation as orthogonal matrix
    transform = end_points['transform'] # BxKxK
    K = transform.get_shape()[1].value
    mat_diff = tf.matmul(transform, tf.transpose(transform, perm=[0,2,1]))
    mat_diff -= tf.constant(np.eye(K), dtype=tf.float32)
    mat_diff_loss = tf.nn.l2_loss(mat_diff) 
    tf.summary.scalar('mat loss', mat_diff_loss)
    print(type(mat_diff_loss))
    return regression_loss + mat_diff_loss * reg_weight

if __name__=='__main__':
    with tf.Graph().as_default():
        inputs = tf.zeros((32,1024,8))
        outputs = get_model(inputs, tf.constant(True))
        gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        print(outputs)
        init = tf.global_variables_initializer()
        with tf.Session() as sess:
            sess.run(init)
            variables = {gvar.op.name: value for gvar, value in zip(gvars, sess.run(gvars))}
            # print(variables)


Overwriting models/pointnet_cls.py


### transform_nets.py

In [42]:
%%writefile models/transform_nets.py
import tensorflow as tf
import numpy as np
import sys
import os
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, '../utils'))
import tf_util

# K is define as the output dimention of transform net
def input_transform_net(point_cloud, is_training, bn_decay=None, K=10):
    """ Input (XYZ) Transform Net, input is BxNx8 gray image
        Return:
            Transformation matrix of size 3xK 
        specificly, input is 8, Transformation matrix is 8xK,K for output dimention
    """
    batch_size = point_cloud.get_shape()[0].value
    num_point = point_cloud.get_shape()[1].value
    feature_dims = point_cloud.get_shape()[2].value # the basic infos of every particle are p_x,p_y,p_y,v_x,v_y,v_z,etc
    input_image = tf.expand_dims(point_cloud, -1)
    # BxNx8x1
    # kernel size will determine the conv direct,(1,8) for vertical and (8,1)for horizon
    net = tf_util.conv2d(input_image, 64, [1,feature_dims],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='tconv1', bn_decay=bn_decay)
    print(net.shape) # BxNx1x64
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='tconv2', bn_decay=bn_decay)
    print(net.shape) # BxNx1x128
    net = tf_util.conv2d(net, 1024, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='tconv3', bn_decay=bn_decay)
    print(net.shape) # BxNx1x1024
    net = tf_util.max_pool2d(net, [num_point,1],
                             padding='VALID', scope='tmaxpool')
    #(32, 1 , 1, 1024)
    net = tf.reshape(net, [batch_size, -1])
    net = tf_util.fully_connected(net, 512, bn=True, is_training=is_training,
                                  scope='tfc1', bn_decay=bn_decay)
    net = tf_util.fully_connected(net, 256, bn=True, is_training=is_training,
                                  scope='tfc2', bn_decay=bn_decay)

    with tf.variable_scope('transform_XYZ') as sc:
#         assert(K==10)
        weights = tf.get_variable('weights', [256, feature_dims*K],
                                  initializer=tf.constant_initializer(0.0),
                                  dtype=tf.float32)
        biases = tf.get_variable('biases', [feature_dims*K],
                                 initializer=tf.constant_initializer(0.0),
                                 dtype=tf.float32)
#         biases += tf.constant([1,0,0,0,1,0,0,0,1], dtype=tf.float32)
        transform = tf.matmul(net, weights)
        transform = tf.nn.bias_add(transform, biases)

    transform = tf.reshape(transform, [batch_size, feature_dims, K])
    print(transform.shape)# (32, 8, 10)
    return transform


def feature_transform_net(inputs, is_training, bn_decay=None, K=64):
    """ Feature Transform Net, input is BxNx1xK
        Return:
            Transformation matrix of size KxK """
    batch_size = inputs.get_shape()[0].value
    num_point = inputs.get_shape()[1].value
    # BxNx1X64
    net = tf_util.conv2d(inputs, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='tconv1', bn_decay=bn_decay)
    print(net.shape) # BxNx1x64
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='tconv2', bn_decay=bn_decay)
    print(net.shape) # BxNx1x128
    net = tf_util.conv2d(net, 1024, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='tconv3', bn_decay=bn_decay)
    # Bx1024x1x1024
    net = tf_util.max_pool2d(net, [num_point,1],
                             padding='VALID', scope='tmaxpool')
    # Bx1x1x1024
    net = tf.reshape(net, [batch_size, -1])
    # Bx1024
    net = tf_util.fully_connected(net, 512, bn=True, is_training=is_training,
                                  scope='tfc1', bn_decay=bn_decay)
    net = tf_util.fully_connected(net, 256, bn=True, is_training=is_training,
                                  scope='tfc2', bn_decay=bn_decay)

    with tf.variable_scope('transform_feat') as sc:
        weights = tf.get_variable('weights', [256, K*K],
                                  initializer=tf.constant_initializer(0.0),
                                  dtype=tf.float32)
        biases = tf.get_variable('biases', [K*K],
                                 initializer=tf.constant_initializer(0.0),
                                 dtype=tf.float32)
        biases += tf.constant(np.eye(K).flatten(), dtype=tf.float32)
        transform = tf.matmul(net, weights)
        transform = tf.nn.bias_add(transform, biases)

    transform = tf.reshape(transform, [batch_size, K, K])
    # Bx64x64
    return transform


Overwriting models/transform_nets.py


In [27]:
%%writefile model_utils.py
import os, argparse

import tensorflow as tf

# The original freeze_graph function
# from tensorflow.python.tools.freeze_graph import freeze_graph 

dir = os.path.dirname(os.path.realpath(__file__))

def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        # We restore the weights
        saver.restore(sess, input_checkpoint)

        # We use a built-in TF helper to export variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the usefull nodes
        ) 

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

    return output_graph_def

def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_dir", type=str, default="log", help="Model folder to export")
    parser.add_argument("--output_node_names", type=str, default="pred", help="The name of the output nodes, comma separated.")
    args = parser.parse_args()

    freeze_graph(args.model_dir, args.output_node_names)

Overwriting model_utils.py


In [53]:
# %run -i models/pointnet_cls.py

## Tensor Bootstrap

In [36]:
"""
If you have trouble in understanding weights of conv layers, read this sampling.
"""
channels = 3
filters_test = np.zeros(shape=(7, 7, channels, 2), dtype=np.float32)

In [37]:
filters_test[:,3,:,0] = 1
filters_test[3,:,:,1] = 1

In [41]:
filters_test[3,:,:,0]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [1., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [37]:
filters_test[:,3,:,0] = 0
filters_test[3,:,:,1]

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [39]:
filters_test[:, 3, :, :]

array([[[1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.],
        [1., 0.]]], dtype=float32)

In [46]:
a = np.array([[[1, 2, 3],[4, 5, 6]],[[2, 4, 6],[9, 5, 6]]])

In [6]:
channels = 3
filters_test = np.zeros(shape=(7, 7, channels, 2), dtype=np.float32)

array([[1, 2, 3],
       [4, 5, 6]])

In [9]:
b = a + 1

In [66]:
c = np.array([1 , 3])
d = np.array([2 , 4])

In [67]:
np.dot(c, d)

14

In [68]:
e = a-b

In [86]:
e

array([[[-1, -1, -1],
        [-1, -1, -1]],

       [[-1, -1, -1],
        [-1, -1, -1]]])

In [71]:
batch_size = a.shape[0]
count = 0
for i in range(batch_size):
    count += np.reduce

In [93]:
np.sum(np.sum(e*e, 2),0).shape

(2,)

In [28]:

import tensorflow as tf
import numpy as np

with tf.Session() as sess:
    a = tf.Variable(5.0, name='a')
    b = tf.Variable(6.0, name='b')
    c = tf.multiply(a, b, name="c")

    sess.run(tf.global_variables_initializer())

    print a.eval() # 5.0
    print b.eval() # 6.0
    print c.eval() # 30.0
    
    tf.train.write_graph(sess.graph_def, 'models/', 'graph.pb', as_text=False)

5.0
6.0
30.0


In [ ]:
np.multiply()

In [11]:
np.multiply(a,b)

array([[[ 2,  6, 12],
        [20, 30, 42]],

       [[ 6, 20, 42],
        [90, 30, 42]]])

In [12]:
channels = 3
filters_test = np.zeros(shape=(7, 7, channels, 2), dtype=np.float32)

array([[[1, 2, 3],
        [4, 5, 6]],

       [[2, 4, 6],
        [9, 5, 6]]])

In [14]:
b.shape

(2, 2, 3)

In [15]:
np.reduce_max(a, axis=2)

AttributeError: 'module' object has no attribute 'reduce_max'

In [44]:
from tensorflow.python.client import device_lib


def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']


get_available_gpus()
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12440206949566615293]

In [50]:
a[[True,False]]

array([[[1, 2, 3],
        [4, 5, 6]]])